In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pandas import Timestamp
from datetime import datetime, timedelta
import time

In [3]:
import pyodbc


In [4]:
server = 'ALAWDREDSQL201.risk.regn.net, 50255'
database = 'red' # enter database name
username = 'dasl_fidoanalytics'
password = 'Jn$a0K29Ty' # add appropriate driver name
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

### Data time range:  c.year_month >= 201901 and c.year_month < 202012
- Not enough dataset for 202012 

In [5]:
start = 202001


In [6]:
end = 202011

In [7]:
%%time
query = ''' 


with base_pol_cte as (select e.year_month,e.ambest_sk, e.coverage_type_sk, e.policy_state_sk,a.groupname, a.shopgroup_sk,
sum(e.cy_exposure) as earned_exposure
from fact_imi_auto_policy_exposure e
inner join dim_imi_ambest a on a.ambest_sk = e.ambest_sk
where cy_date_sk >=20190101 and cy_date_sk <= 20201231
group by e.year_month,e.ambest_sk,e.coverage_type_sk, e.policy_state_sk,a.groupname, a.shopgroup_sk),



base_clm_wd_cte as (select w.year_month,w.ambest_sk,w.coverage_type_sk, w.policy_state_sk,w.window_sk,a.groupname, a.shopgroup_sk,
sum(w.claim_reported_count) as claim_reported_count_window,
sum(w.claim_paid_count) as paid_count_window,
sum(w.claim_paid_closed_count) as claim_paid_closed_count_window, 
sum(w.daystoclose_adj) as daystoclose_adj_window,
sum(w.daystopay_adj) as daystopay_adj_window,
sum(w.paid_closed_claim_amount) as paid_closed_claim_amount_window
from fact_imi_auto_claim_window w
left join dim_imi_ambest a on a.ambest_sk = w.ambest_sk
where w.cy_date_sk >=20190101 and w.cy_date_sk <= 20201231
group by w.year_month,w.ambest_sk,w.coverage_type_sk, w.policy_state_sk,w.window_sk,a.groupname, a.shopgroup_sk)


select w.year_month,w.policy_state_sk, w.coverage_type_sk, w.shopgroup_sk, w.window_sk, s.shopgroup_name, C.coverage_desc, g.state_province_cd,
sum(e.earned_exposure) as earned_exposure , 
sum(w.claim_paid_closed_count_window) as claim_paid_closed_count_window, 
sum(w.claim_reported_count_window) as claim_reported_count_window , 
sum(w.daystoclose_adj_window) as daystoclose_adj_window,
sum(w.daystopay_adj_window) as daystopay_adj_window,
sum(paid_closed_claim_amount_window) as paid_closed_claim_amount_window,
sum(w.paid_count_window) as paid_count_window
from base_pol_cte e
FULL OUTER JOIN base_clm_wd_cte w on w.ambest_sk = e.ambest_sk and w.coverage_type_sk = e.coverage_type_sk and w.policy_state_sk = e.policy_state_sk and w.shopgroup_sk = e.shopgroup_sk
FULL OUTER JOIN dim_imi_auto_coverage_type c ON  w.coverage_type_sk = c.coverage_type_sk
FULL OUTER JOIN  dim_geography_v g ON w.policy_state_sk = g.geography_sk
FULL OUTER JOIN dim_imi_shopgroup S ON W.shopgroup_sk = s.shopgroup_sk
where w.year_month >= {} and w.year_month <= {}
and w.coverage_type_sk in (1,2,3,8)
group by w.year_month,w.policy_state_sk, w.coverage_type_sk, w.shopgroup_sk,w.window_sk, s.shopgroup_name, c.coverage_desc, g.state_province_cd





''';


CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.63 µs


In [8]:
%%time
query = query.format(start, end)
data = pd.read_sql(query, cnxn)
data.head()

CPU times: user 1.88 s, sys: 168 ms, total: 2.04 s
Wall time: 35.7 s


,year_month,policy_state_sk,coverage_type_sk,shopgroup_sk,window_sk,shopgroup_name,coverage_desc,state_province_cd,earned_exposure,claim_paid_closed_count_window,claim_reported_count_window,daystoclose_adj_window,daystopay_adj_window,paid_closed_claim_amount_window,paid_count_window
0,202010,49,3,75,3,COUNTRY MUTUAL,Comprehensive,WI,45834.657488,1872,2328,34584.0,34584.0,3497112,1896
1,202001,40,8,157,2,LIBERTY MUTUAL,Property Damage,RI,62839.246591,744,3528,9732.0,9384.0,2752824,864
2,202001,3,1,193,4,NATIONWIDE,Bodily Injury,AR,77635.125975,168,600,6024.0,5328.0,710088,216
3,202007,16,3,16,3,ALFA VISION,Comprehensive,IN,15671.967127,216,264,4692.0,4692.0,498048,264
4,202004,18,8,193,3,NATIONWIDE,Property Damage,KY,87478.175244,672,1752,16812.0,15972.0,2401584,768


In [9]:
cnxn.close()  

In [10]:
data.shape

(242817, 15)

In [11]:
min(data['year_month'])

202001

In [12]:
max(data['year_month'])

202011

### change the year_month format 
- 201909 to 2019-09-01

In [13]:
df = pd.DataFrame(data['year_month'].tolist(), columns = ['year_month_list']) 
df['year_month_str'] = df['year_month_list'].astype(str) 
df['year_month_format'] = df['year_month_str'].apply(lambda x: datetime.strptime(x, '%Y%m'))
data['year_month_format'] = df['year_month_format']
max_date = max(data['year_month_format'])
data['max_date'] = max_date

In [14]:
def month_diff(a, b):
    return 12 * (a.dt.year - b.dt.year) + (a.dt.month - b.dt.month)

In [15]:
months_diff = month_diff(data['max_date'], data['year_month_format'])
data['months_diff'] = months_diff

In [16]:
prior = ['Prior-Month-']*data.shape[0]    
date_reverse_str = months_diff.astype(str)        
data['month_label'] = prior + date_reverse_str
data['month_label'] [data['month_label'] =='Prior-Month-0'] = 'Current-Month'


<ipython-input-16-57f2461080e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['month_label'] [data['month_label'] =='Prior-Month-0'] = 'Current-Month'


## Check data quality

In [17]:
cols = [
'year_month_format',
'month_label',
'claim_reported_count_window',
'paid_closed_claim_amount_window',
'claim_paid_closed_count_window',
'earned_exposure',
'policy_state_sk',
'coverage_type_sk',
'shopgroup_sk',
'window_sk',
'shopgroup_name',
'coverage_desc',
'state_province_cd']

In [18]:
data2 = data[cols]

## Check $0 claim paid amount and the 0 claim count

In [19]:
zero_claims= data2[(data2['paid_closed_claim_amount_window'] ==0) ]

In [20]:
zero_claims.shape

(31829, 13)

In [21]:
print('The percent of the claims have either $0 claim amount or 0 claim count',round(len(zero_claims) / len(data2) *100,0),'%')

The percent of the claims have either $0 claim amount or 0 claim count 13.0 %


## Need to exclude the data with $0 paid claim amount and 0 paid counts 

In [22]:
good_data = data2[(data2['paid_closed_claim_amount_window'] >0) & (data2['claim_paid_closed_count_window'] >0)]

In [23]:
good_data.shape

(210988, 13)

In [24]:

Current_Month_units = pd.DataFrame(good_data[good_data['month_label'] == 'Current-Month'].pivot_table(index=['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd'],
               margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['paid_closed_claim_amount_window','claim_paid_closed_count_window','claim_reported_count_window', 'earned_exposure'],
                aggfunc={'paid_closed_claim_amount_window':np.sum,
                       'claim_paid_closed_count_window':np.sum,
                        'claim_reported_count_window':np.sum,
                        'earned_exposure':np.sum}))
Current_Month_units.columns= [ 'claim_paid_closed_count_window_cur_mos','claim_reported_count_window_cur_mos', 'earned_exposure_cur_mos','paid_closed_claim_amount_window_cur_mos']
Current_Month_units['paid_closed_severity_window_cur_mos'] =  Current_Month_units['paid_closed_claim_amount_window_cur_mos']/Current_Month_units['claim_paid_closed_count_window_cur_mos']
Current_Month_units['reported_frequency_window_cur_mos'] =  Current_Month_units['claim_reported_count_window_cur_mos']/Current_Month_units['earned_exposure_cur_mos']


In [25]:
prior_month1_units = pd.DataFrame(good_data[good_data['month_label'] == 'Prior-Month-1'].pivot_table(index=['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd'],
                  margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['paid_closed_claim_amount_window','claim_paid_closed_count_window','claim_reported_count_window','earned_exposure'],
               aggfunc={'paid_closed_claim_amount_window':np.sum,
                       'claim_paid_closed_count_window':np.sum,
                        'claim_reported_count_window':np.sum,
                        'earned_exposure':np.sum}))
prior_month1_units.columns= ['claim_paid_closed_count_window_prior_1mos', 'claim_reported_count_window_prior_1mos','earned_exposure_prior_1mos','paid_closed_claim_amount_prior_1mos']
prior_month1_units['paid_closed_severity_window_prior_1mos'] =  prior_month1_units['paid_closed_claim_amount_prior_1mos']/prior_month1_units['claim_paid_closed_count_window_prior_1mos']
prior_month1_units['reported_frequency_window_prior_1mos'] =  prior_month1_units['claim_reported_count_window_prior_1mos']/prior_month1_units['earned_exposure_prior_1mos']


In [26]:
prior_month2_units = pd.DataFrame(good_data[good_data['month_label'] == 'Prior-Month-2'].pivot_table(index=['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd'],
                  margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['paid_closed_claim_amount_window','claim_paid_closed_count_window','claim_reported_count_window','earned_exposure'],
               aggfunc={'paid_closed_claim_amount_window':np.sum,
                       'claim_paid_closed_count_window':np.sum,
                        'claim_reported_count_window':np.sum,
                        'earned_exposure':np.sum}))
prior_month2_units.columns= ['claim_paid_closed_count_window_prior_2mos', 'claim_reported_count_window_prior_2mos','earned_exposure_prior_2mos','paid_closed_claim_amount_prior_2mos']
prior_month2_units['paid_closed_severity_window_prior_2mos'] =  prior_month2_units['paid_closed_claim_amount_prior_2mos']/prior_month2_units['claim_paid_closed_count_window_prior_2mos']
prior_month2_units['reported_frequency_window_prior_2mos'] =  prior_month2_units['claim_reported_count_window_prior_2mos']/prior_month2_units['earned_exposure_prior_2mos']





In [27]:
prior_month3_units = pd.DataFrame(good_data[good_data['month_label'] == 'Prior-Month-3'].pivot_table(index=['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd'],
                  margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['paid_closed_claim_amount_window','claim_paid_closed_count_window','claim_reported_count_window','earned_exposure'],
               aggfunc={'paid_closed_claim_amount_window':np.sum,
                       'claim_paid_closed_count_window':np.sum,
                        'claim_reported_count_window':np.sum,
                        'earned_exposure':np.sum}))
prior_month3_units.columns= ['claim_paid_closed_count_window_prior_3mos', 'claim_reported_count_window_prior_3mos','earned_exposure_prior_3mos','paid_closed_claim_amount_prior_3mos']
prior_month3_units['paid_closed_severity_window_prior_3mos'] =  prior_month3_units['paid_closed_claim_amount_prior_3mos']/prior_month3_units['claim_paid_closed_count_window_prior_3mos']
prior_month3_units['reported_frequency_window_prior_3mos'] =  prior_month3_units['claim_reported_count_window_prior_3mos']/prior_month3_units['earned_exposure_prior_3mos']







In [28]:
prior_month4_units = pd.DataFrame(good_data[good_data['month_label'] == 'Prior-Month-4'].pivot_table(index=['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd'],
                  margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['paid_closed_claim_amount_window','claim_paid_closed_count_window','claim_reported_count_window','earned_exposure'],
               aggfunc={'paid_closed_claim_amount_window':np.sum,
                       'claim_paid_closed_count_window':np.sum,
                        'claim_reported_count_window':np.sum,
                        'earned_exposure':np.sum}))
prior_month4_units.columns= ['claim_paid_closed_count_window_prior_4mos', 'claim_reported_count_window_prior_4mos','earned_exposure_prior_4mos','paid_closed_claim_amount_prior_4mos']
prior_month4_units['paid_closed_severity_window_prior_4mos'] =  prior_month4_units['paid_closed_claim_amount_prior_4mos']/prior_month4_units['claim_paid_closed_count_window_prior_4mos']
prior_month4_units['reported_frequency_window_prior_4mos'] =  prior_month4_units['claim_reported_count_window_prior_4mos']/prior_month4_units['earned_exposure_prior_4mos']









In [29]:
prior_month5_units = pd.DataFrame(good_data[good_data['month_label'] == 'Prior-Month-5'].pivot_table(index=['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd'],
                  margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['paid_closed_claim_amount_window','claim_paid_closed_count_window','claim_reported_count_window','earned_exposure'],
               aggfunc={'paid_closed_claim_amount_window':np.sum,
                       'claim_paid_closed_count_window':np.sum,
                        'claim_reported_count_window':np.sum,
                        'earned_exposure':np.sum}))
prior_month5_units.columns= ['claim_paid_closed_count_window_prior_5mos', 'claim_reported_count_window_prior_5mos','earned_exposure_prior_5mos','paid_closed_claim_amount_prior_5mos']
prior_month5_units['paid_closed_severity_window_prior_5mos'] =  prior_month5_units['paid_closed_claim_amount_prior_5mos']/prior_month5_units['claim_paid_closed_count_window_prior_5mos']
prior_month5_units['reported_frequency_window_prior_5mos'] =  prior_month5_units['claim_reported_count_window_prior_5mos']/prior_month5_units['earned_exposure_prior_5mos']





In [30]:
prior_month6_units = pd.DataFrame(good_data[good_data['month_label'] == 'Prior-Month-6'].pivot_table(index=['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd'],
                  margins=False,
               #margins_name='total',  # defaults to 'All'
                values=['paid_closed_claim_amount_window','claim_paid_closed_count_window','claim_reported_count_window','earned_exposure'],
               aggfunc={'paid_closed_claim_amount_window':np.sum,
                       'claim_paid_closed_count_window':np.sum,
                        'claim_reported_count_window':np.sum,
                        'earned_exposure':np.sum}))
prior_month6_units.columns= ['claim_paid_closed_count_window_prior_6mos', 'claim_reported_count_window_prior_6mos','earned_exposure_prior_6mos','paid_closed_claim_amount_prior_6mos']
prior_month6_units['paid_closed_severity_window_prior_6mos'] =  prior_month6_units['paid_closed_claim_amount_prior_6mos']/prior_month6_units['claim_paid_closed_count_window_prior_6mos']
prior_month6_units['reported_frequency_window_prior_6mos'] =  prior_month6_units['claim_reported_count_window_prior_6mos']/prior_month6_units['earned_exposure_prior_6mos']



In [31]:
frames = [Current_Month_units, prior_month1_units, prior_month2_units, prior_month3_units,prior_month4_units,prior_month5_units, prior_month6_units]

In [32]:
result = pd.concat(frames,axis=1, sort=False)

### Check Number of NA for severity

## Replace NA to 0
- The reason of NA appear: for per pilicy_state_sk, coverage_type, shopgroup_sk and name, there's no claim paid amounts/counts for certain months 

In [33]:
result2 = pd.DataFrame(result).fillna(0)
result2.head()
final_result = result2

## Add alerts for severity

In [34]:
final_result['avg_severity_6mos'] =  final_result[['paid_closed_severity_window_prior_1mos',
                                                   'paid_closed_severity_window_prior_2mos',
                                                   'paid_closed_severity_window_prior_3mos',
                                                   'paid_closed_severity_window_prior_4mos',
                                                   'paid_closed_severity_window_prior_5mos',
                                                  'paid_closed_severity_window_prior_6mos']].mean(axis=1)

In [35]:
final_result['std_severity_6mos'] = final_result[['paid_closed_severity_window_prior_1mos',
                                                  'paid_closed_severity_window_prior_2mos',
                                                  'paid_closed_severity_window_prior_3mos',
                                                  'paid_closed_severity_window_prior_4mos', 
                                                  'paid_closed_severity_window_prior_5mos',
                                                 'paid_closed_severity_window_prior_6mos']].std(axis=1)

#### calculating upper-bound and lower-bound for severity. 
- If the lower bound < 0, then set lower-bound = 0

In [36]:
final_result['severity_threshold_up'] = final_result['avg_severity_6mos'] + 6.5*final_result['std_severity_6mos']

In [37]:
severity_threshold_low = final_result['avg_severity_6mos'] - 5*final_result['std_severity_6mos']
final_result['severity_threshold_low'] = severity_threshold_low.apply(lambda x: 0 if x < 0 else x)


### Add alerts to Frequency 

In [38]:
final_result['avg_freq_6mos'] =  final_result[['reported_frequency_window_cur_mos',
'reported_frequency_window_prior_1mos',
'reported_frequency_window_prior_2mos',
'reported_frequency_window_prior_3mos',
'reported_frequency_window_prior_4mos',
'reported_frequency_window_prior_5mos',
'reported_frequency_window_prior_6mos']].mean(axis=1)

In [39]:
final_result['std_freq_6mos'] =  final_result[['reported_frequency_window_cur_mos',
'reported_frequency_window_prior_1mos',
'reported_frequency_window_prior_2mos',
'reported_frequency_window_prior_3mos',
'reported_frequency_window_prior_4mos',
'reported_frequency_window_prior_5mos',
'reported_frequency_window_prior_5mos',
 'reported_frequency_window_prior_6mos']].std(axis=1)

In [40]:
final_result['freq_threshold_up'] = final_result['avg_freq_6mos'] + 2.2*final_result['std_freq_6mos']

In [41]:
freq_threshold_low = final_result['avg_freq_6mos'] - 2.2*final_result['std_freq_6mos']
final_result['freq_threshold_low'] = freq_threshold_low.apply(lambda x: 0 if x < 0 else x)

## Add alerts for the reported counts 

In [42]:
final_result['sum_report_counts_3mos'] =  final_result[['claim_reported_count_window_prior_1mos',
                                                        'claim_reported_count_window_prior_2mos',
                                                        'claim_reported_count_window_prior_3mos']].sum(axis=1)

In [43]:
final_result['sum_report_counts_6mos'] =  final_result[['claim_reported_count_window_prior_1mos',
                                                        'claim_reported_count_window_prior_2mos',
                                                        'claim_reported_count_window_prior_3mos',
                                                        'claim_reported_count_window_prior_4mos',
                                                        'claim_reported_count_window_prior_5mos',
                                                       'claim_reported_count_window_prior_6mos']].sum(axis=1)

In [44]:
final_result['avg_report_counts_6mos'] = final_result[['claim_reported_count_window_prior_1mos',
                                                       'claim_reported_count_window_prior_2mos',
                                                       'claim_reported_count_window_prior_3mos',
                                                       'claim_reported_count_window_prior_4mos',
                                                       'claim_reported_count_window_prior_5mos',
                                                      'claim_reported_count_window_prior_6mos']].mean(axis=1)

In [45]:
final_result['std_report_counts_6mos'] = final_result[['claim_reported_count_window_prior_1mos',
                                                       'claim_reported_count_window_prior_2mos',
                                                       'claim_reported_count_window_prior_3mos',
                                                       'claim_reported_count_window_prior_4mos',
                                                       'claim_reported_count_window_prior_5mos',
                                                      'claim_reported_count_window_prior_6mos']].std(axis=1)

In [46]:
final_result['report_counts_threshold_up'] = final_result['avg_report_counts_6mos'] + 5*final_result['std_report_counts_6mos'] 

In [47]:
report_counts_threshold_low= final_result['avg_report_counts_6mos'] - 5*final_result['std_report_counts_6mos'] 
final_result['report_counts_threshold_low'] = report_counts_threshold_low.apply(lambda x: 0 if x < 0 else x)

## Add alerts for paid_closed_counts 

In [48]:
final_result['sum_closed_counts_3mos'] =  final_result[[
'claim_paid_closed_count_window_prior_1mos', 
'claim_paid_closed_count_window_prior_2mos', 
'claim_paid_closed_count_window_prior_3mos']].sum(axis=1)

In [49]:
final_result['sum_closed_counts_6mos'] =  final_result[['claim_paid_closed_count_window_prior_1mos', 
'claim_paid_closed_count_window_prior_2mos', 
'claim_paid_closed_count_window_prior_3mos', 
'claim_paid_closed_count_window_prior_4mos', 
'claim_paid_closed_count_window_prior_5mos',
 'claim_paid_closed_count_window_prior_6mos']].sum(axis=1)

In [50]:
final_result['avg_closed_counts_6mos'] =  final_result[['claim_paid_closed_count_window_prior_1mos', 
'claim_paid_closed_count_window_prior_2mos', 
'claim_paid_closed_count_window_prior_3mos', 
'claim_paid_closed_count_window_prior_4mos', 
'claim_paid_closed_count_window_prior_5mos',
 'claim_paid_closed_count_window_prior_6mos']].mean(axis=1)

In [51]:
final_result['std_closed_counts_6mos'] =  final_result[['claim_paid_closed_count_window_prior_1mos', 
'claim_paid_closed_count_window_prior_2mos', 
'claim_paid_closed_count_window_prior_3mos', 
'claim_paid_closed_count_window_prior_4mos', 
'claim_paid_closed_count_window_prior_5mos',
'claim_paid_closed_count_window_prior_6mos']].std(axis=1)

In [52]:
final_result['closed_counts_threshold_up'] = final_result['avg_closed_counts_6mos'] + 5*final_result['std_closed_counts_6mos'] 

In [53]:
closed_counts_threshold_low= final_result['avg_closed_counts_6mos'] - 5*final_result['std_closed_counts_6mos'] 
final_result['closed_counts_threshold_low'] = closed_counts_threshold_low.apply(lambda x: 0 if x < 0 else x)

In [54]:
## Add shift attribute to check the flutuate percentage 

In [55]:
final_result['severity_shift'] = (final_result['paid_closed_severity_window_cur_mos'] - final_result['avg_severity_6mos'])/final_result['avg_severity_6mos']

In [56]:
final_result['reported_frequency_shift'] = (final_result['reported_frequency_window_cur_mos'] - final_result['avg_freq_6mos'])/final_result['avg_freq_6mos']

In [57]:
final_result['report_counts_shift'] = (final_result['claim_reported_count_window_cur_mos'] - final_result['avg_report_counts_6mos'])/final_result['avg_report_counts_6mos']

In [58]:
final_result['closed_counts_shift'] = (final_result['claim_paid_closed_count_window_cur_mos'] - final_result['avg_closed_counts_6mos'])/final_result['avg_closed_counts_6mos']

## Check all the results 

In [59]:
round(final_result[['sum_closed_counts_6mos','sum_report_counts_6mos']].describe(), 2)

,sum_closed_counts_6mos,sum_report_counts_6mos
count,25392.00,25392.00
mean,18055.69,37233.27
std,78955.18,145322.00
min,0.00,0.00
25%,96.00,240.00
50%,939.50,2400.00
75%,5777.25,13056.00
max,1830576.00,2764584.00


In [60]:
sum_closed_counts_6mos_median = final_result[['sum_closed_counts_6mos']].median()[0]

In [61]:
sum_closed_counts_6mos_mean = final_result[['sum_closed_counts_6mos']].mean()[0]*0.5

In [62]:
sum_report_counts_6mos_mean = final_result[['sum_report_counts_6mos']].mean()[0]*0.5

In [63]:
sum_report_counts_6mos_median = final_result[['sum_report_counts_6mos']].median()[0]

### set up floors for closed counts and reported counts

In [64]:
good_result  = final_result[(final_result['sum_closed_counts_6mos'] >sum_closed_counts_6mos_mean) & (final_result['sum_report_counts_6mos'] >sum_report_counts_6mos_mean)]

In [65]:
#good_result  = final_result[(final_result['sum_closed_counts_6mos'] >sum_closed_counts_6mos_median) & (final_result['sum_report_counts_6mos'] >sum_report_counts_6mos_median)]

### check alerts for closed_counts

In [66]:
len(good_result[good_result['claim_paid_closed_count_window_cur_mos']>good_result['closed_counts_threshold_up']])

8

In [67]:
len(good_result[(good_result['claim_paid_closed_count_window_cur_mos']<good_result['closed_counts_threshold_low']) &(good_result['claim_paid_closed_count_window_cur_mos']>0) ])

54

In [68]:
len(good_result[good_result['claim_reported_count_window_cur_mos']>good_result['report_counts_threshold_up']])

7

In [69]:
len(good_result[(good_result['claim_reported_count_window_cur_mos']<good_result['report_counts_threshold_low']) &(good_result['claim_reported_count_window_cur_mos']>0) ])

21

In [70]:
len(good_result[good_result['paid_closed_severity_window_cur_mos'] >good_result['severity_threshold_up']])

30

In [71]:
len(good_result[(good_result['paid_closed_severity_window_cur_mos'] <good_result['severity_threshold_low']) & good_result['paid_closed_severity_window_cur_mos'] >0 ])

9

In [72]:
len(good_result[good_result['reported_frequency_window_cur_mos'] > good_result['freq_threshold_up']])

3

In [73]:
len(good_result[(good_result['reported_frequency_window_cur_mos'] < good_result['freq_threshold_low']) & good_result['reported_frequency_window_cur_mos'] >0 ])

14

In [74]:
alerts = good_result[good_result['paid_closed_severity_window_cur_mos'] >good_result['severity_threshold_up']]

In [75]:
#alerts.to_csv('good_result_window_alerts.csv', sep='\t', encoding='utf-8')

In [76]:
good_final_result = good_result.reset_index()

### Add alerts lables

In [77]:
closed_counts_alerts = []
for i in range(len(good_final_result)):
    if good_final_result['claim_paid_closed_count_window_cur_mos'][i] > good_final_result['closed_counts_threshold_up'][i] :
        results = 'high-closed-counts_alerts'
    elif (good_final_result['claim_paid_closed_count_window_cur_mos'][i] < good_final_result['closed_counts_threshold_low'][i]) & (good_final_result['claim_paid_closed_count_window_cur_mos'][i] >0):
        results = 'low-closed-counts_alerts'
    else:
        results = 'no-alerts'
    closed_counts_alerts.append(results)
good_final_result['closed_counts_alerts'] = closed_counts_alerts

In [78]:
if (len(good_final_result[good_final_result['closed_counts_alerts'] =='high-closed-counts_alerts']) > 20):
    top_closed_counts_alerts_index = good_final_result[good_final_result['closed_counts_alerts'] =='high-closed-counts_alerts'] \
                    .sort_values(by=['closed_counts_shift'], ascending=False).index[:10]
    top_closed_counts_alerts_index
    for index in top_closed_counts_alerts_index:
        closed_counts_alerts[index] = 'top-high-closed-counts_alerts'
        good_final_result['closed_counts_alerts'] = closed_counts_alerts
else:
    pass

In [79]:
if (len(good_final_result[good_final_result['closed_counts_alerts'] =='low-closed-counts_alerts']) > 20):
    top_closed_counts_alerts_index = good_final_result[good_final_result['closed_counts_alerts'] =='low-closed-counts_alerts'] \
                    .sort_values(by=['closed_counts_shift']).index[:10]
    top_closed_counts_alerts_index
    for index in top_closed_counts_alerts_index:
        closed_counts_alerts[index] = 'top-low-closed-counts_alerts'
        good_final_result['closed_counts_alerts'] = closed_counts_alerts
else:
    pass

In [80]:
good_final_result['closed_counts_alerts'].value_counts()

no-alerts                       4456
low-closed-counts_alerts          44
top-low-closed-counts_alerts      10
high-closed-counts_alerts          8
Name: closed_counts_alerts, dtype: int64

In [81]:
reported_counts_alerts = []
for i in range(len(good_final_result)):
    if good_final_result['claim_reported_count_window_cur_mos'][i] > good_final_result['report_counts_threshold_up'][i] :
        results = 'high-reported-counts_alerts'
    elif (good_final_result['claim_reported_count_window_cur_mos'][i] < good_final_result['report_counts_threshold_low'][i]) & (good_final_result['claim_reported_count_window_cur_mos'][i] >0):
        results = 'low-reported-counts_alerts'
    else:
        results = 'no-alerts'
    reported_counts_alerts.append(results)
good_final_result['reported_counts_alerts'] = reported_counts_alerts

In [82]:
if (len(good_final_result[good_final_result['reported_counts_alerts'] =='high-reported-counts_alerts']) > 20):
    top_reported_counts_alerts_index = good_final_result[good_final_result['reported_counts_alerts'] =='high-reported-counts_alerts'] \
                    .sort_values(by=['report_counts_shift'], ascending=False).index[:10]
    top_reported_counts_alerts_index
    for index in top_reported_counts_alerts_index:
        reported_counts_alerts[index] = 'top-high-reported-counts_alerts'
        good_final_result['reported_counts_alerts'] = reported_counts_alerts
else:
    pass


In [83]:
if (len(good_final_result[good_final_result['reported_counts_alerts'] =='low-reported-counts_alerts']) > 20):
    top_reported_counts_alerts_index = good_final_result[good_final_result['reported_counts_alerts'] =='low-reported-counts_alerts'] \
                    .sort_values(by=['report_counts_shift']).index[:10]
    top_reported_counts_alerts_index
    for index in top_reported_counts_alerts_index:
        reported_counts_alerts[index] = 'top-low-reported-counts_alerts'
        good_final_result['reported_counts_alerts'] = reported_counts_alerts
else:
    pass


In [84]:
good_final_result['reported_counts_alerts'].value_counts()

no-alerts                         4490
low-reported-counts_alerts          11
top-low-reported-counts_alerts      10
high-reported-counts_alerts          7
Name: reported_counts_alerts, dtype: int64

In [85]:
severity_alerts = []
for i in range(len(good_final_result)):
    if good_final_result['paid_closed_severity_window_cur_mos'][i] > good_final_result['severity_threshold_up'][i] :
        results = 'high-severity-alerts'
    elif (good_final_result['paid_closed_severity_window_cur_mos'][i] < good_final_result['severity_threshold_low'][i]) & (good_final_result['paid_closed_severity_window_cur_mos'][i] >0):
        results = 'low-severity-alerts'
    else:
        results = 'no-alerts'
    severity_alerts.append(results)
good_final_result['severity_alerts'] = severity_alerts

In [86]:
if (len(good_final_result[good_final_result['severity_alerts'] =='high-severity-alerts']) > 20):
    top_severity_alerts_index = good_final_result[good_final_result['severity_alerts'] =='high-severity-alerts'] \
                    .sort_values(by=['severity_shift'], ascending=False).index[:10]
    for index in top_severity_alerts_index:
        severity_alerts[index] = 'top-high-severity-alerts'
        good_final_result['severity_alerts'] = severity_alerts
else:
    pass

In [87]:
if (len(good_final_result[good_final_result['severity_alerts'] =='low-severity-alerts']) > 20):
    top_severity_alerts_index = good_final_result[good_final_result['severity_alerts'] =='low-severity-alerts'] \
                    .sort_values(by=['severity_shift']).index[:10]
    for index in top_severity_alerts_index:
        severity_alerts[index] = 'top-low-severity-alerts'
        good_final_result['severity_alerts'] = severity_alerts
else:
    pass


In [88]:
good_final_result['severity_alerts'].value_counts()

no-alerts                   4479
high-severity-alerts          20
top-high-severity-alerts      10
low-severity-alerts            9
Name: severity_alerts, dtype: int64

In [89]:
freq_alerts = []
for i in range(len(good_final_result)):
    if good_final_result['reported_frequency_window_cur_mos'][i] > good_final_result['freq_threshold_up'][i] :
        results = 'high-freq-alerts'
    elif (good_final_result['reported_frequency_window_cur_mos'][i] < good_final_result['freq_threshold_low'][i]) & (good_final_result['reported_frequency_window_cur_mos'][i] >0):
        results = 'low-freq-alerts'
    else:
        results = 'no-alerts'
    freq_alerts.append(results)
good_final_result['frequency_alerts'] = freq_alerts

In [90]:
if (len(good_final_result[good_final_result['frequency_alerts'] =='high-freq-alerts']) > 20):
    top_freq_alerts_index = good_final_result[good_final_result['freq_alerts'] =='high-freq-alerts'] \
                    .sort_values(by=['reported_frequency_shift'], ascending=False).index[:10]
    for index in top_freq_alerts_index:
        freq_alerts[index] = 'top-high-freq-alerts'
        good_final_result['frequency_alerts'] = freq_alerts
else:
    pass

In [91]:
if (len(good_final_result[good_final_result['frequency_alerts'] =='low-freq-alerts']) > 20):
    top_freq_alerts_index = good_final_result[good_final_result['freq_alerts'] =='low-freq-alerts'] \
                    .sort_values(by=['reported_frequency_shift']).index[:10]
    for index in top_freq_alerts_index:
        freq_alerts[index] = 'top-low-freq-alerts'
        good_final_result['frequency_alerts'] = freq_alerts
else:
    pass

In [92]:
good_final_result['frequency_alerts'].value_counts()

no-alerts           4501
low-freq-alerts       14
high-freq-alerts       3
Name: frequency_alerts, dtype: int64

In [93]:
import sys
print(sys.path)

['/home/autumn/fido-models', '/opt/anaconda3/lib/python38.zip', '/opt/anaconda3/lib/python3.8', '/opt/anaconda3/lib/python3.8/lib-dynload', '', '/home/autumn/.local/lib/python3.8/site-packages', '/home/autumn/jupyterhub-ssh', '/opt/anaconda3/lib/python3.8/site-packages', '/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions', '/home/autumn/.ipython']


In [94]:
### export alerts seperately 

In [94]:
# reported_counts = good_final_result[['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd',
# 'report_counts_shift',                                  
# 'claim_reported_count_window_cur_mos', 
# 'claim_reported_count_window_prior_1mos', 
# 'claim_reported_count_window_prior_2mos', 
# 'claim_reported_count_window_prior_3mos', 
# 'claim_reported_count_window_prior_4mos', 
# 'claim_reported_count_window_prior_5mos', 
# 'claim_reported_count_window_prior_6mos', 
# 'sum_report_counts_3mos', 
# 'sum_report_counts_6mos', 
# 'avg_report_counts_6mos', 
# 'std_report_counts_6mos', 
# 'report_counts_threshold_up', 
# 'report_counts_threshold_low', 
# 'reported_counts_alerts']]
# reported_counts.to_csv('/home/autumn/contributory/data_results/reported_counts_alerts_202011.csv', sep='\t', encoding='utf-8')

In [95]:
# closed_counts = good_final_result[['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd',
# 'closed_counts_shift',            
# 'claim_paid_closed_count_window_cur_mos',
# 'claim_paid_closed_count_window_prior_1mos',
# 'claim_paid_closed_count_window_prior_2mos',
# 'claim_paid_closed_count_window_prior_3mos',
# 'claim_paid_closed_count_window_prior_4mos',
# 'claim_paid_closed_count_window_prior_5mos',
# 'claim_paid_closed_count_window_prior_6mos',
# 'sum_closed_counts_3mos',
# 'sum_closed_counts_6mos',
# 'avg_closed_counts_6mos',
# 'std_closed_counts_6mos',
# 'closed_counts_threshold_up',
# 'closed_counts_threshold_low',
# 'closed_counts_alerts']]
# closed_counts.to_csv('/home/autumn/contributory/data_results/closed_counts_alerts_202011.csv', sep='\t', encoding='utf-8')

In [96]:
# severity = good_final_result[['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd',
#  'severity_shift',                              
# 'paid_closed_severity_window_cur_mos',
# 'paid_closed_severity_window_prior_1mos',
# 'paid_closed_severity_window_prior_2mos',
# 'paid_closed_severity_window_prior_3mos',
# 'paid_closed_severity_window_prior_4mos',
# 'paid_closed_severity_window_prior_5mos',
# 'paid_closed_severity_window_prior_6mos',
# 'avg_severity_6mos',
# 'std_severity_6mos',
# 'severity_threshold_up',
# 'severity_threshold_low',
# 'severity_alerts']]
# severity.to_csv('/home/autumn/contributory/data_results/severity_alerts_202011.csv', sep='\t', encoding='utf-8')

In [97]:
# frequency = good_final_result[['policy_state_sk','coverage_type_sk','shopgroup_sk','window_sk','shopgroup_name','coverage_desc','state_province_cd',
# 'reported_frequency_shift',
# 'reported_frequency_window_cur_mos',
# 'reported_frequency_window_prior_1mos',
# 'reported_frequency_window_prior_2mos',
# 'reported_frequency_window_prior_3mos',
# 'reported_frequency_window_prior_4mos',
# 'reported_frequency_window_prior_5mos',
# 'reported_frequency_window_prior_6mos',                              
# 'avg_freq_6mos',
# 'std_freq_6mos',
# 'freq_threshold_up',
# 'freq_threshold_low',
# 'frequency_alerts']]
# frequency.to_csv('/home/autumn/contributory/data_results/frequency_alerts_202011.csv', sep='\t', encoding='utf-8')

# Wrtie the results back to SQL server 

In [98]:
good_final_result['alert_year_month'] = str(end)

In [99]:
good_final_result.shape

(4518, 78)

In [106]:
for col in good_final_result:
    print(col)

policy_state_sk
coverage_type_sk
shopgroup_sk
window_sk
shopgroup_name
coverage_desc
state_province_cd
claim_paid_closed_count_window_cur_mos
claim_reported_count_window_cur_mos
earned_exposure_cur_mos
paid_closed_claim_amount_window_cur_mos
paid_closed_severity_window_cur_mos
reported_frequency_window_cur_mos
claim_paid_closed_count_window_prior_1mos
claim_reported_count_window_prior_1mos
earned_exposure_prior_1mos
paid_closed_claim_amount_prior_1mos
paid_closed_severity_window_prior_1mos
reported_frequency_window_prior_1mos
claim_paid_closed_count_window_prior_2mos
claim_reported_count_window_prior_2mos
earned_exposure_prior_2mos
paid_closed_claim_amount_prior_2mos
paid_closed_severity_window_prior_2mos
reported_frequency_window_prior_2mos
claim_paid_closed_count_window_prior_3mos
claim_reported_count_window_prior_3mos
earned_exposure_prior_3mos
paid_closed_claim_amount_prior_3mos
paid_closed_severity_window_prior_3mos
reported_frequency_window_prior_3mos
claim_paid_closed_count_wind

In [100]:
contributory_data = good_final_result.values.tolist()

In [101]:
server = 'ALAWDREDSQL201.risk.regn.net, 50255'
database = 'red' # enter database name
username = 'dasl_fidoanalytics'
password = 'Jn$a0K29Ty' # add appropriate driver name
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [107]:
cursor.execute("DROP TABLE IF EXISTS contributory_alerts")

table_sql = ("CREATE TABLE contributory_alerts( policy_state_sk real, \
coverage_type_sk real, \
shopgroup_sk real, \
window_sk real, \
shopgroup_name text, \
coverage_desc text, \
state_province_cd text, \
report_counts_shift real, \
closed_counts_shift real, \
severity_shift real, \
reported_frequency_shift real, \
claim_paid_closed_count_window_cur_mos real, \
claim_reported_count_window_cur_mos real, \
earned_exposure_cur_mos real, \
paid_closed_claim_amount_window_cur_mos real, \
paid_closed_severity_window_cur_mos real, \
reported_frequency_window_cur_mos real, \
claim_paid_closed_count_window_prior_1mos real, \
claim_reported_count_window_prior_1mos real, \
earned_exposure_prior_1mos real, \
paid_closed_claim_amount_prior_1mos real, \
paid_closed_severity_window_prior_1mos real, \
reported_frequency_window_prior_1mos real, \
claim_paid_closed_count_window_prior_2mos real, \
claim_reported_count_window_prior_2mos real, \
earned_exposure_prior_2mos real, \
paid_closed_claim_amount_prior_2mos real, \
paid_closed_severity_window_prior_2mos real, \
reported_frequency_window_prior_2mos real, \
claim_paid_closed_count_window_prior_3mos real, \
claim_reported_count_window_prior_3mos real, \
earned_exposure_prior_3mos real, \
paid_closed_claim_amount_prior_3mos real, \
paid_closed_severity_window_prior_3mos real, \
reported_frequency_window_prior_3mos real, \
claim_paid_closed_count_window_prior_4mos real, \
claim_reported_count_window_prior_4mos real, \
earned_exposure_prior_4mos real, \
paid_closed_claim_amount_prior_4mos real, \
paid_closed_severity_window_prior_4mos real, \
reported_frequency_window_prior_4mos real, \
claim_paid_closed_count_window_prior_5mos real, \
claim_reported_count_window_prior_5mos real, \
earned_exposure_prior_5mos real, \
paid_closed_claim_amount_prior_5mos real, \
paid_closed_severity_window_prior_5mos real, \
reported_frequency_window_prior_5mos real, \
claim_paid_closed_count_window_prior_6mos real, \
claim_reported_count_window_prior_6mos real, \
earned_exposure_prior_5mos real, \
paid_closed_claim_amount_prior_6mos real, \
paid_closed_severity_window_prior_6mos real, \
reported_frequency_window_prior_6mos real, \
avg_severity_6mos real, \
std_severity_6mos real, \
severity_threshold_up real, \
severity_threshold_low real, \
avg_freq_6mos real, \
std_freq_6mos real, \
freq_threshold_up real, \
freq_threshold_low real, \
sum_report_counts_3mos real, \
sum_report_counts_6mos real, \
avg_report_counts_6mos real, \
std_report_counts_6mos real, \
report_counts_threshold_up real, \
report_counts_threshold_low real, \
sum_closed_counts_3mos real, \
sum_closed_counts_6mos real, \
avg_closed_counts_6mos real, \
std_closed_counts_6mos real, \
closed_counts_threshold_up real, \
closed_counts_threshold_low real, \
closed_counts_alerts text, \
reported_counts_alerts text, \
severity_alerts text, \
frequency_alerts text, \
alert_year_month ntext)")
cursor.execute(table_sql)
cnxn.commit()

ProgrammingError: ('42S21', "[42S21] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column names in each table must be unique. Column name 'earned_exposure_prior_5mos' in table 'contributory_alerts' is specified more than once. (2705) (SQLExecDirectW)")

In [102]:
def write_data(contributory_data):
    for row in contributory_data:
        data_info = row
        cursor.execute("""INSERT INTO contributory_alerts values {}""".format("(" +"?,"*71+"?)"), data_info)
    
    cnxn.commit()

In [105]:
write_data(contributory_data)

ProgrammingError: ('The SQL contains 72 parameter markers, but 78 parameters were supplied', 'HY000')

In [ ]:
cnxn.close()  